# Humans or Horses binary classifier with small dataset

In [6]:
from tensorflow.python.keras.optimizer_v2.rmsprop import RMSProp
!pip install scipy # for data augmentation

ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [7]:
import tensorflow as tf
import numpy as np
from scipy.ndimage import zoom
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [9]:
training_dir = "images/train"
validation_dir = "images/validation"

In [10]:
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [11]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [12]:
model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')
])

C:\Users\sumes\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
)

33/33 ━━━━━━━━━━━━━━━━━━━━ 23s 705ms/step - accuracy: 0.9406 - loss: 0.1423 - val_accuracy: 0.6172 - val_loss: 4.4757


In [18]:
from tensorflow.keras.utils import load_img, img_to_array

paths = [
    "images/test/sample1.jpg",
    "images/test/sample2.jpg",
    "images/test/sample3.jpg",
    "images/test/sample4.jpg",
    "images/test/sample5.jpg"
]

target_size = (300, 300)

batch = []
for p in paths:
    img = load_img(p, target_size=target_size)
    arr = img_to_array(img)
    batch.append(arr)

batch = np.stack(batch, axis=0)  # shape: (N, 300, 300, 3)
preds = model.predict(batch)     # shape: (N, num_outputs)

# Example: binary classifier with sigmoid output
for path, pred in zip(paths, preds):
    # pred is a scalar or shape (1,) for binary models
    score = float(pred if np.ndim(pred) == 0 else pred[0])
    label = "humans" if score > 0.5 else "horses"
    print(f"{path}: score={score:.3f} → {label}")


images/test/sample1.jpg: score=1.000 → humans
images/test/sample2.jpg: score=1.000 → humans
images/test/sample3.jpg: score=1.000 → humans
images/test/sample4.jpg: score=1.000 → humans
images/test/sample5.jpg: score=1.000 → humans


In [16]:
import numpy as np
from keras.preprocessing import image
path = "images/test/sample3.jpg"
x = image.load_img(path, target_size=(300, 300))
x = image.img_to_array(x)
x = np.expand_dims(x, axis=0)

image_tensor = np.vstack([x])
classes = model.predict(image_tensor)
print(classes)
print(classes[0])
if classes[0]>0.5:
    print(path + " is a human")
else:
    print(path + " is a horse")

[[1.]]
[1.]
images/test/sample3.jpg is a human
